# Amber Molecular Dynamics Simulation
Amber Molecular Dynamics Simulation of PDB protein and solvent OPC
* Laboratorio de diseño de fármacos Nanocell
* https://www.nanocell.cl
* Dr. Alexis Salas Burgos
* v.1
* October 2021

Adapted from http://ambermd.org/tutorials/basic/tutorial0

* AMBER stands for Assisted Model Building and Energy Refinement. It refers not only to the molecular dynamics programs, but also a set of force fields that describe the potential energy function and parameters of the interactions of biomolecules.

In order to run a Molecular Dynamics simulation in Amber, each molecule's interactions are described by a molecular force field. The force field has specific parameters defined for each molecule.

*sander* is the basic MD engine of Amber. *pmemd* is the high performance implementation of the MD engine that contains a subset of features of sander. pmemd can also be run with acceleration from graphics processing units (GPU).

In order to run an MD simulation with sander or pmemd, three key files are needed:
* prmtop - The file that describes the parameter and topology of the molecules in the system
* inpcrd - The file that describes the initial molecular coordinates of the system
* mdin - The file that describes the settings for the Amber MD engine

# Amber
https://ambermd.org
## Requirements
* test_version = 2021
* conda create AmberTools21 -c conda-forge ambertools=21 compilers
* conda activate AmberTools21

# 1. Prepare the Ensembl

In [2]:
# Preparing
import os
#neccesary to collect the output
pdb_code = '1AKI'
os.chdir("../run")
new_dir = "{0}_amber".format(pdb_code)
try:os.mkdir(new_dir)
except:pass
os.chdir(new_dir)
cmd1 = "wget https://files.rcsb.org/download/{0}.pdb".format(pdb_code)
cmd2 = "grep -v HOH {0}.pdb > {0}_clean.pdb".format(pdb_code)
cmd3 = "conda activate AmberTools21 && packmol-memgen --pdb {0}_clean.pdb --solvate --cubic --dist 15 --salt \
--saltcon 0.15 --minimize  --ffwat opc --ffprot ff19SB --overwrite && conda deactivate".format(pdb_code)

In [24]:
# Running
os.system(cmd1)
os.system(cmd2)
#To running in the run folder from the terminal
print(cmd3)

conda activate AmberTools21 && packmol-memgen --pdb 1AKI_clean.pdb --solvate --cubic --dist 15 --salt --saltcon 0.15 --minimize  --ffwat opc --ffprot ff19SB --overwrite && conda deactivate


In [47]:
with open("min.in", 'r') as f:
    import re
    texto = f.read()
    pattern = "restraintmask.+:((\d+)-(\d+))"
    m = re.findall(pattern, texto)
start = m[0][1]
end   = m[0][2]

In [48]:
min1 = "01min_wat.in"

texto = """Stage 1 - minimisation of ubiquitin:
initial minimisation solvent + ions
&cntrl
 imin = 1,
 maxcyc = 1000,
 ncyc = 500,
 ntpr=100,
 ntb = 1,
 ntr = 1,
 cut = 10
/
Hold the protein fixed
500.0
RES {0} {1}
END
END
""".format(start,end)

with open(min1, "w") as f:
    f.write(texto)

In [49]:
min2 = "02min_wat.in"

texto = """Stage 2 - minimisation of ubiquitin unrestricted:
initial minimisation whole system
&cntrl
imin = 1,
maxcyc = 2500,
ncyc = 1000,
ntb = 1,
ntr = 0,
cut = 10
/
"""

with open(min2, "w") as f:
    f.write(texto)

In [50]:
md1 = "03md_wat.in"

texto = """Stage 3 - Molecular Dynamics Simulation restricted:
20ps MD with res on protein
&cntrl
 imin = 0,
 irest = 0,
 ntx = 1,
 ntb = 1,
 cut = 10,
 ntr = 1,
 ntc = 2,
 ntf = 2,
 tempi = 0.0,
 temp0 = 300.0,
 ntt = 3,
 gamma_ln = 1.0,
 nstlim = 10000, dt = 0.002,
 ntpr = 100, ntwx = 500, ntwr = 1000
/
Keep protein fixed with weak restraints
10.0
RES {0} {1}
END
END
""".format(start,end)

with open(md1, "w") as f:
    f.write(texto)

In [4]:
%%writefile 04md_wat_100ps.in

Stage 4 - Molecular Dynamics Simulation of ubiquitin:
100ps MD
&cntrl
 imin = 0, irest = 1, ntx = 7,
 ntxo = 2    ! formato de restart rst 1:rst, 2: netcdf
 ioutfm = 1  ! set mdcr trj to nc (netcdf)
 ntb = 2, pres0 = 1.0, ntp = 1,
 taup = 2.0,
 cut = 10, ntr = 0,
 ntc = 2, ntf = 2,
 tempi = 300.0, temp0 = 300.0,
 ntt = 3, gamma_ln = 1.0,
 nstlim = 50000, dt = 0.002,
 ntpr = 100, ntwx = 500, ntwr = 1000
/

Overwriting 04md_wat_100ps.in


# 2. Running local PC

In [5]:
def run_command(pdb_code, time='100ps', cluster=False):
    """create a script to running amber"""
    
    if cluster == True:
        integrator = "pmemd.cuda"
        bash_file = """#!/bin/bash
#SBATCH -J MDS{0}
#SBATCH --partition=defq
#SBATCH --nodes=1
#SBATCH --tasks-per-node=20
#SBATCH --gres=gpu:4

#Note: in one node pmemd with mpi and cuda is more slow that only cuda
#Use only pmemd.cuda
module load mdynamics/amber/20.20\n\n
""".format(pdb_code)

    else:
        integrator = "sander"
        bash_file = "#!/bin/bash"

    cmd1 = "{0} -O -i 01min_wat.in -o 01min_wat.out -p solvated_{1}_clean_wat.top -c solvated_{1}_clean_wat.crd -r 01min_wat.rst -ref solvated_{1}_clean_wat.crd -inf 01min_wat.info\n".format(integrator, pdb_code)
    cmd2 = "{0} -O -i 02min_wat.in -o 02min_wat.out -p solvated_{1}_clean_wat.top -c 01min_wat.rst -r 02min_wat.rst -inf 02min_wat.info\n".format(integrator, pdb_code)
    cmd3 = "{0} -O -i 03md_wat.in -o 03md_wat.out -p solvated_{1}_clean_wat.top -c 02min_wat.rst -r 03md_wat.rst -x 03md_wat.mdcrd -ref 02min_wat.rst -inf 03md_wat.info\n".format(integrator, pdb_code)
    cmd4 = "{0} -O -i 04md_wat_{2}.in -o 04md_wat_{2}.out -p solvated_{1}_clean_wat.top -c 03md_wat.rst -r 04md_wat_{2}.rst -x 04md_wat_{2}.mdcrd -ref 03md_wat.rst -inf 04md_wat_{2}.info\n".format(integrator, pdb_code, time)
    
    bash_file=bash_file+cmd1+cmd2+cmd3+cmd4
    with open("amber_{0}_{1}.sch".format(pdb_code, time),"w") as f:
        f.write(bash_file)
        print("amber_{0}_{1}.sch wrote!!".format(pdb_code, time))
    return "amber_{0}_{1}.sch".format(pdb_code, time)

In [9]:
import os
print ("cd", os.getcwd())
print ("conda activate AmberTools21")
run_command(pdb_code, time='100ps', cluster=False)

cd /Users/alsalas/GDrive/Github/Labcodes/Dynamics/Molecular_Dynamics_Simulation/run/1AKI_amber
conda activate AmberTools21
amber_1AKI_100ps.sch wrote!!


# 3. Running in GPU Cluster

In [6]:
%%writefile 04md_wat_10ns.in

Stage 4 - Molecular Dynamics Simulation of ubiquitin:
100ps MD
&cntrl
 imin = 0, irest = 1, ntx = 7,
 ntxo = 2    ! formato de restart rst 1:rst, 2: netcdf
 ioutfm = 1  ! set mdcr trj to nc (netcdf)
 ntb = 2, pres0 = 1.0, ntp = 1,
 taup = 2.0,
 cut = 10, ntr = 0,
 ntc = 2, ntf = 2,
 tempi = 300.0, temp0 = 300.0,
 ntt = 3, gamma_ln = 1.0,
 nstlim = 5000000, dt = 0.002,
 ntpr = 100, ntwx = 500, ntwr = 1000
/

Overwriting 04md_wat_10ns.in


In [7]:
slurm_file = run_command(pdb_code, time='10ns', cluster=True)

amber_1AKI_10ns.sch wrote!!


In [1]:
def create_dir(nombre="Amber"):
    from time import strftime
    import os
    fecha = strftime("%Y%j_%H%M%S")
    directorio_pdbs = fecha + "_" + nombre
    os.mkdir(directorio_pdbs)
    return directorio_pdbs

def compress(directorio):
    import tarfile
    outfile = "%s.tar.gz" % directorio
    tar = tarfile.open(outfile, "w:gz")
    tar.add(directorio, arcname=directorio)
    tar.close()
    return outfile

In [8]:
import shutil
import os
#recuerda que debes crear este archivo python con el cuaderno 00
import conection
user = conection.user
ipbender  = conection.ipbender
ipcluster = conection.ipcluster

directorio = create_dir()
files = ['01min_wat.in','02min_wat.in','03md_wat.in','04md_wat_10ns.in','solvated_{0}_clean_wat.top'.format(pdb_code),
         'solvated_{0}_clean_wat.crd'.format(pdb_code), slurm_file]
for f in files:
    shutil.copy(f, directorio)
out_compress_folder = compress(directorio)
print("cd", os.getcwd())
print("scp -rC {0} {1}:".format(out_compress_folder, ipbender))
print("ssh {0}@{1}".format(user, ipbender))
print("scp -rC {0} {1}:".format(out_compress_folder, ipcluster))
print("ssh {0}@{1}".format(user, ipcluster))
print("tar -xzvf {0}".format(out_compress_folder))
print("cd {0}".format(out_compress_folder[:-7]))
print("sbatch", slurm_file)

cd /Users/alsalas/GDrive/Github/Labcodes/Dynamics/Molecular_Dynamics_Simulation/run/1AKI_amber
scp -rC 2021304_132456_Amber.tar.gz 152.00.00.00:
ssh alsalas@152.00.00.00
scp -rC 2021304_132456_Amber.tar.gz 152.00.00.00:
ssh alsalas@152.00.00.00
tar -xzvf 2021304_132456_Amber.tar.gz
cd 2021304_132456_Amber
sbatch amber_1AKI_10ns.sch
